## Step 1: Initialize the Database


In [16]:
# Run init_db.py to create the SQLite schema
!python ../init_db.py  # Adjust the path if needed


DB path: /Users/zacseidel/Documents/GitHub/momentum-screener/data/market_data.sqlite
Will create directory: /Users/zacseidel/Documents/GitHub/momentum-screener/data
Database initialized at /Users/zacseidel/Documents/GitHub/momentum-screener/data/market_data.sqlite


## Step 2: Connect and Inspect Tables

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/market_data.sqlite")

# Check that all tables exist
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)


,name
0,index_constituents
1,index_allocations
2,daily_prices
3,top10_picks
4,company_info
5,company_news


## Step 3 Preview the Table Schemas

In [18]:
# Show column structure for each table
def describe_table(table):
    return pd.read_sql(f"PRAGMA table_info({table})", conn)

describe_table("index_constituents")


,cid,name,type,notnull,dflt_value,pk
0,0,ticker,TEXT,1,None,1
1,1,company,TEXT,0,None,0
2,2,index_type,TEXT,0,None,2
3,3,gics_sector,TEXT,0,None,0
4,4,gics_sub_industry,TEXT,0,None,0
5,5,headquarters,TEXT,0,None,0
6,6,date_first_added,TEXT,0,None,0
7,7,founded,TEXT,0,None,0
8,8,date_added,DATE,1,DATE('now'),3


## Step 4: Scrape and Insert Initial Index Constituents

In [31]:
import pandas as pd
import requests
from io import StringIO

def get_index_constituents(index="sp500"):
    if index == "sp500":
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    elif index == "sp400":
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_400_companies"
    else:
        raise ValueError("Index must be 'sp500' or 'sp400'")

    response = requests.get(url)
    response.raise_for_status()
    html = response.text

    df = pd.read_html(StringIO(html))[0]  # <- Wrapped in StringIO now
    df.columns = [col.lower().strip() for col in df.columns]

    rename_map = {
        "symbol": "ticker",
        "security": "company",
        "gics sector": "gics_sector",
        "gics sub-industry": "gics_sub_industry",
        "headquarters location": "headquarters",
        "date added": "date_added",
        "founded": "founded"
    }

    available_renames = {k: v for k, v in rename_map.items() if k in df.columns}
    df = df.rename(columns=available_renames)

    for col in ["date_added", "founded"]:
        if col not in df.columns:
            df[col] = None

    df["index_type"] = index
    df["date_added"] = pd.Timestamp.today().date()

    return df.loc[:, [
        "ticker", "company", "index_type", "gics_sector",
        "gics_sub_industry", "headquarters", "date_added",
        "founded"
    ]]


## Step 5: Load into SQLite

In [33]:
# Load and insert S&P 500 and S&P 400
df_sp500 = get_index_constituents("sp500")
df_sp400 = get_index_constituents("sp400")

df_sp500.to_sql("index_constituents", conn, if_exists="replace", index=False)
df_sp400.to_sql("index_constituents", conn, if_exists="replace", index=False)


401

## Confirm it worked

In [34]:
pd.read_sql("SELECT * FROM index_constituents ORDER BY date_added DESC LIMIT 10", conn)


,ticker,company,index_type,gics_sector,gics_sub_industry,headquarters,date_added,founded
0,AA,Alcoa,sp400,Materials,Aluminum,"Pittsburgh, Pennsylvania",2025-05-08,None
1,AAL,American Airlines Group,sp400,Industrials,Passenger Airlines,"Fort Worth, Texas",2025-05-08,None
2,AAON,AAON,sp400,Industrials,Building Products,"Tulsa, Oklahoma",2025-05-08,None
3,ACHC,Acadia Healthcare,sp400,Health Care,Health Care Facilities,"Franklin, Tennessee",2025-05-08,None
4,ACI,Albertsons,sp400,Consumer Staples,Food Retail,"Boise, Idaho",2025-05-08,None
5,ACM,AECOM,sp400,Industrials,Construction & Engineering,"Dallas, Texas",2025-05-08,None
6,ADC,Agree Realty,sp400,Real Estate,Retail REITs,"Bloomfield Hills, Michigan",2025-05-08,None
7,AFG,American Financial Group,sp400,Financials,Multi-line Insurance,"Cincinnati, Ohio",2025-05-08,None
8,AGCO,AGCO,sp400,Industrials,Agricultural & Farm Machinery,"Duluth, Georgia",2025-05-08,None
9,AIT,Applied Industrial Technologies,sp400,Industrials,Trading Companies & Distributors,"Cleveland, Ohio",2025-05-08,None


## Adding Allocations

In [43]:
import sys
import os
import importlib

# Ensure project root is in the path
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

# Import and reload your module
import src.allocations
importlib.reload(src.allocations)

# Now you can call the updated function
from src.allocations import update_index_allocations
update_index_allocations()


Saved file: spy_holdings.xlsx
Attempting to connect to database at: /Users/zacseidel/Documents/GitHub/momentum-screener/data/market_data.sqlite
Columns in spy_holdings.xlsx: ['Name', 'Ticker', 'Identifier', 'SEDOL', 'Weight', 'Sector', 'Shares Held', 'Local Currency']
Stored 504 rows for sp500
Saved file: mdy_holdings.xlsx
Attempting to connect to database at: /Users/zacseidel/Documents/GitHub/momentum-screener/data/market_data.sqlite
Columns in mdy_holdings.xlsx: ['Name', 'Ticker', 'Identifier', 'SEDOL', 'Weight', 'Sector', 'Shares Held', 'Local Currency']
Stored 402 rows for sp400
